In [107]:
# import dependecies
import numpy
import sys
import nltk
nltk.download('stopwords')
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [70]:
# load data
# loading data and opening our input in the form of a text file
# project Gutenberg files
file = open('frankenstein.txt').read()

In [71]:
# tokenisation
# standardization
# What is Tokenisation? Tkenisation is the process pf breaking a stream of text up into words phrases symbols or other
# meaningful elements
def tokenize_words(input):
    # lowercase everything to standardise it
    input = input.lower()
    # installing the tokenizer
    tokenizer = RegexpTokenizer(r'\w+')
    # tokenizing the text into tokens
    tokens = tokenizer.tokenize(input)
    # filtering the stopwords using lambda
    filtered = filter(lambda token: token not in stopwords.words('english'), tokens)
    return "".join(filtered)

# preprocess the input data, make tokens
processed_inputs = tokenize_words(file)

In [72]:
# chars to numbers
# convert characters in our input to numbers
# we'll sort the list of the set of all characets that appear in out i/p text and then use the enumerate fn
# to get numbers that represent the characters
# We'll then create a dictonary that stores the keys and values, or the characters and the numbers that represent them
chars = sorted(list(set(processed_inputs)))
char_to_num = dict((c, i) for i, c in enumerate(chars))

In [73]:
# check if words to chars to chars to num(?) has worked?
# just so we get an idea of whether out process of converting words to characters has worked
# we just print the length of out varaibles
input_len = len(processed_inputs)
vocab_len = len(chars)
print("Total number of characters:", input_len)
print("Total Vocab:", vocab_len)

Total number of characters: 241901
Total Vocab: 42


In [74]:
# seq length
# we're defining how long we want an individual sequence here
# an individual sequence is a complete mapping of input characters as integers
seq_length = 100
x_data = []
y_data = []

In [75]:
# loop through the sequence
# here we're going through the entire list of i/ps and converting the chars to numbers with a for loop
# this will create a bunch of sequences where each sequence starts with the next character in the i/p data
# beginning with the first character
for i in range(0, input_len - seq_length, 1):
    # define i/p and o/p sequences
    # i/p is the current character plus the desired sequence length
    in_seq = processed_inputs[i:i + seq_length]
    # out sequence is the initital character plus total sequence length
    out_seq = processed_inputs[i + seq_length]
    # converting the list of characters to integers based on previous values and appending the values to our lists
    x_data.append([char_to_num[char]for char in in_seq])
    y_data.append([char_to_num[out_seq]])

# check to see how many input sequences we have
n_patterns = len(x_data)
print("Totla Patterns:", n_patterns)

Totla Patterns: 241801


In [76]:
# convert input sequence to np array that our network can use
X = numpy.reshape(x_data, (n_patterns, seq_length, 1))
X = X/float(vocab_len)

In [77]:
# one hot-encoding our label data
y = np_utils.to_categorical(y_data)

In [110]:
# creating the model
# creating a sequenctial model
# droput is used to prevent overfitting
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

AttributeError: module 'tensorflow' has no attribute 'get_default_graph'

In [ ]:
# compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [55]:
# saving weights
filepath = 'model_weights_saved.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose= 1, save_best_only=True, mode='min')
desired_callbacks = [checkpoint]

In [ ]:
# fit model and let it train
model.fit(X, y, epochs=4, batch_size=256, callbacks=deleted_callbacks)

In [ ]:
# random seed to help generate
start = numpy.random.randint(0, len(x_data) - 1)
pattern = x_data[start]
print("Random Seed: ")
print("\".".join([num_to_char[value] for value in pattern]), "\"")

In [ ]:
# generate the text
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x/float(vocab_len)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = num_to_char[index]
    seq_in = [num_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]